In [4]:
import datetime as dt
import logging
import os
import sys
import boto3
import pytz
#from awsglue.utils import getResolvedOptions

# Obtener parámetros del trabajo
#args = getResolvedOptions(
#    sys.argv, ['JOB_NAME', 'S3_STAGE_PREFIX', 'DYNAMO_CONFIG_TABLE', 'DYNAMO_ENDPOINT_TABLE', 
#                'ENDPOINT', 'PROCESS_ID', 'ARN_ROLE_CRAWLER', 'PROJECT_NAME', 'TEAM', 'DATA_SOURCE'])
##################################
boto3.setup_default_session(profile_name='prd-valorx-admin', region_name='us-east-1')

In [ ]:
glue_crawler_manager = GlueCrawlerManager(logger)
job_role_arn = glue_crawler_manager.get_job_role_arn('sofia-dev-datalake-crawler_stage-job')

2025-05-18 14:32:44,384 sofia-dev-datalake-crawler_stage-job DEBUG Rol ARN obtenido: arn:aws:iam::566121885938:role/CdkDatalakeIngestBigMagic-sofiadevdatalakecrawlerst-Av8iwB3ELbnA


In [12]:
glue_crawler_manager.grant_lf_tag_permissions(job_role_arn)

2025-05-18 14:33:17,561 sofia-dev-datalake-crawler_stage-job ERROR Error al conceder permisos LF-Tag: An error occurred (AccessDeniedException) when calling the GrantPermissions operation: Resource does not exist or requester is not authorized to access requested permissions.


False

In [14]:
import datetime as dt
import logging
import os
import sys
import boto3
import pytz
#from awsglue.utils import getResolvedOptions

# Obtener parámetros del trabajo
#args = getResolvedOptions(
#    sys.argv, ['JOB_NAME', 'S3_STAGE_PREFIX', 'DYNAMO_CONFIG_TABLE', 'DYNAMO_ENDPOINT_TABLE', 
#                'ENDPOINT', 'PROCESS_ID', 'ARN_ROLE_CRAWLER', 'PROJECT_NAME', 'TEAM', 'DATA_SOURCE'])
##################################
boto3.setup_default_session(profile_name='prd-valorx-admin', region_name='us-east-1')
##################################
args = {
    'JOB_NAME': 'sofia-dev-datalake-crawler_stage-job',
    'S3_STAGE_PREFIX': 's3://sofia-566121885938-us-east-1-dev-datalake-stage-s3/',
    'DYNAMO_CONFIG_TABLE': 'sofia-dev-datalake-configuration-ddb',
    'DYNAMO_ENDPOINT_TABLE': 'sofia-dev-datalake-credentials-ddb',
    'ENDPOINT': 'PEBDDATA',
    'PROCESS_ID': '10',
    'ARN_ROLE_CRAWLER': 'arn:aws:iam::566121885938:role/sofia-dev-datalake-crawler_stage-role',
    'PROJECT_NAME': 'datalake',
    'TEAM': 'sofia',
    'DATA_SOURCE': 'bigmagic'    
}
# Obtener parámetros del trabajo
#args = getResolvedOptions(
#    sys.argv, ['JOB_NAME', 'S3_STAGE_PREFIX', 'DYNAMO_CONFIG_TABLE', 'DYNAMO_ENDPOINT_TABLE', 
#                'ENDPOINT', 'PROCESS_ID', 'ARN_ROLE_CRAWLER', 'PROJECT_NAME', 'TEAM', 'DATA_SOURCE'])

# Configuración del logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Clase utilitaria para gestionar recursos de AWS Glue
class GlueCrawlerManager:
    """Clase utilitaria para gestionar recursos de AWS Glue"""
    
    def __init__(self, logger=None):
        """Inicializar la clase con servicios AWS"""
        self.logger = logger or logging.getLogger(__name__)
        
        try:
            self.dynamodb = boto3.resource('dynamodb')
            self.client_glue = boto3.client('glue')
            self.client_lakeformation = boto3.client('lakeformation')
        except Exception as e:
            self.logger.error(f"Error al inicializar servicios AWS: {str(e)}")
            raise

    def create_database(self, database_name):
        """Crear base de datos en el catálogo de datos"""
        try:
            self.client_glue.create_database(
                DatabaseInput={
                    'Name': database_name
                }
            )
            self.logger.info(f"Base de datos {database_name} creada exitosamente")
            return True
        except Exception as e:
            self.logger.error(f"Error al crear la base de datos {database_name}: {str(e)}")
            return False

    def database_exists(self, database_name):
        """Verificar si existe la base de datos en el catálogo"""
        try:
            self.client_glue.get_database(
                Name=database_name
            )
            self.logger.info(f"Base de datos {database_name} encontrada")
            return True
        except self.client_glue.exceptions.EntityNotFoundException:
            self.logger.info(f"Base de datos {database_name} no encontrada")
            return False
        except Exception as e:
            self.logger.error(f"Error al verificar la base de datos {database_name}: {str(e)}")
            return False

    def get_job_role_arn(self, job_name):
        """Obtener el ARN del rol del trabajo"""
        try:
            role = self.client_glue.get_job(JobName=job_name)['Job']['Role']
            self.logger.info(f"Rol ARN obtenido: {role}")
            return role
        except Exception as e:
            self.logger.error(f"Error al obtener el rol ARN del trabajo {job_name}: {str(e)}")
            return None

    def grant_database_permissions(self, role_arn, database_name):
        """Asignar permisos de la base de datos en Lake Formation"""
        try:
            self.client_lakeformation.grant_permissions(
                Principal={
                    'DataLakePrincipalIdentifier': role_arn
                },
                Resource={
                    'Database': {
                        'Name': database_name
                    },
                },
                Permissions=[
                    'ALL',
                ],
                PermissionsWithGrantOption=[
                    'ALL',
                ]
            )
            self.logger.info(f"Permisos concedidos a {role_arn} sobre {database_name}")
            return True
        except Exception as e:
            self.logger.error(f"Error al conceder permisos en Lake Formation: {str(e)}")
            return False

    def grant_lf_tag_permissions(self, role_arn, tag_key="Level", tag_values=["Stage"]):
        """Asignar permisos de etiquetas LF al rol"""
        try:
            self.client_lakeformation.grant_permissions(
                Principal={
                    'DataLakePrincipalIdentifier': role_arn
                },
                Resource={
                    'LFTag': {
                        'TagKey': tag_key,
                        'TagValues': tag_values
                    },
                },
                Permissions=[
                    'ASSOCIATE',
                ],
                PermissionsWithGrantOption=[
                    'ASSOCIATE',
                ]
            )
            self.logger.info(f"Permisos LF-Tag concedidos a {role_arn}")
            return True
        except Exception as e:
            self.logger.error(f"Error al conceder permisos LF-Tag: {str(e)}")
            return False

    def add_lf_tags_to_database(self, database_name, tag_key="Level", tag_values=["Stage"]):
        """Añadir etiquetas LF a la base de datos"""
        try:
            self.client_lakeformation.add_lf_tags_to_resource(
                Resource={
                    'Database': {
                        'Name': database_name
                    },
                },
                LFTags=[
                    {
                        'TagKey': tag_key,
                        'TagValues': tag_values
                    },
                ]
            )
            self.logger.info(f"Etiquetas LF añadidas a {database_name}")
            return True
        except Exception as e:
            self.logger.error(f"Error al añadir etiquetas LF a la base de datos: {str(e)}")
            return False

    def create_delta_crawler(self, crawler_name, role_arn, database_name, delta_targets):
        """Crear crawler para objetivos Delta"""
        try:
            if not delta_targets:
                self.logger.warning("No hay objetivos para crear el crawler")
                return False
                
            self.client_glue.create_crawler(
                Name=crawler_name,
                Role=role_arn,
                DatabaseName=database_name,
                Targets={
                    'DeltaTargets': delta_targets
                }
            )
            self.logger.info(f"Crawler {crawler_name} creado exitosamente")
            return True
        except Exception as e:
            self.logger.error(f"Error al crear crawler {crawler_name}: {str(e)}")
            return False

    def update_delta_crawler(self, crawler_name, role_arn, database_name, delta_targets):
        """Actualizar crawler existente"""
        try:
            if not delta_targets:
                self.logger.warning("No hay objetivos para actualizar el crawler")
                return False
                
            self.client_glue.update_crawler(
                Name=crawler_name,
                Role=role_arn,
                DatabaseName=database_name,
                Targets={
                    'DeltaTargets': delta_targets
                }
            )
            self.logger.info(f"Crawler {crawler_name} actualizado exitosamente")
            return True
        except Exception as e:
            self.logger.error(f"Error al actualizar crawler {crawler_name}: {str(e)}")
            return False

    def crawler_exists(self, crawler_name):
        """Verificar si existe el crawler"""
        try:
            self.client_glue.get_crawler(
                Name=crawler_name
            )
            self.logger.info(f"Crawler {crawler_name} encontrado")
            return True
        except self.client_glue.exceptions.EntityNotFoundException:
            self.logger.info(f"Crawler {crawler_name} no encontrado")
            return False
        except Exception as e:
            self.logger.error(f"Error al verificar crawler {crawler_name}: {str(e)}")
            return False

    def start_crawler(self, crawler_name):
        """Iniciar el crawler"""
        try:
            # Verificar si el crawler ya está en ejecución
            crawler_info = self.client_glue.get_crawler(Name=crawler_name)
            crawler_state = crawler_info.get('Crawler', {}).get('State')
            
            if crawler_state == 'RUNNING':
                self.logger.info(f"El crawler {crawler_name} ya está en ejecución")
                return True
                
            self.client_glue.start_crawler(
                Name=crawler_name
            )
            self.logger.info(f"Crawler {crawler_name} iniciado exitosamente")
            return True
        except Exception as e:
            self.logger.error(f"Error al iniciar crawler {crawler_name}: {str(e)}")
            return False

    def update_dynamodb_attribute(self, table_name, key_name, key_value, attr_name, attr_value):
        """Actualizar atributo en DynamoDB"""
        try:
            self.logger.info(f'Actualizando DynamoDB: {key_name}={key_value}, {attr_name}={attr_value}, tabla={table_name}')
            dynamo_table = self.dynamodb.Table(table_name)
            response = dynamo_table.update_item(
                Key={key_name: key_value},
                AttributeUpdates={
                    attr_name: {
                        'Value': attr_value,
                        'Action': 'PUT'
                    }
                }
            )
            return True
        except Exception as e:
            self.logger.error(f"Error al actualizar DynamoDB: {str(e)}")
            return False

    def batch_update_dynamodb_attributes(self, table_name, items_to_update):
        """Actualizar múltiples atributos en DynamoDB usando batch write"""
        try:
            if not items_to_update:
                return True
                
            self.logger.info(f'Actualizando {len(items_to_update)} items en DynamoDB con batch write')
            
            # Implementar lógica de batch_write_item
            # Nota: batch_write_item tiene límite de 25 items por llamada
            batch_size = 25
            for i in range(0, len(items_to_update), batch_size):
                batch = items_to_update[i:i+batch_size]
                request_items = {
                    table_name: [
                        {
                            'PutRequest': {
                                'Item': item
                            }
                        } for item in batch
                    ]
                }
                self.dynamodb.batch_write_item(RequestItems=request_items)
                
            return True
        except Exception as e:
            self.logger.error(f"Error en batch update DynamoDB: {str(e)}")
            return False

    def get_dynamodb_item(self, table_name, key_name, key_value):
        """Obtener un item específico de DynamoDB"""
        try:
            dynamo_table = self.dynamodb.Table(table_name)
            response = dynamo_table.get_item(Key={key_name: key_value})
            return response.get('Item')
        except Exception as e:
            self.logger.error(f"Error al obtener item de DynamoDB: {str(e)}")
            return None

    def get_dynamodb_items(self, table_name):
        """Obtener todos los items de una tabla DynamoDB"""
        try:
            dynamo_table = self.dynamodb.Table(table_name)
            response = dynamo_table.scan()
            items = response['Items']
            
            # Manejar paginación si hay más de 1MB de datos
            while 'LastEvaluatedKey' in response:
                response = dynamo_table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
                items.extend(response['Items'])
                
            return items
        except Exception as e:
            self.logger.error(f"Error al obtener items de DynamoDB: {str(e)}")
            return []

# Funciones del flujo de trabajo personalizado
def get_tables_and_prepare_delta_targets(glue_crawler_manager, config_table, endpoint_name, s3_target, team, data_source):
    """Obtener las tablas y preparar targets en una sola función optimizada"""
    delta_targets = []
    empty_tables = []
    total_list = []
    tables_to_update = []
    
    try:
        # Una sola llamada a DynamoDB para obtener todas las tablas
        all_items = glue_crawler_manager.get_dynamodb_items(config_table)
        
        # Filtrar localmente por endpoint
        endpoint_items = [item for item in all_items if item.get('ENDPOINT') == endpoint_name]
        
        for item in endpoint_items:
            table_name = item.get('TARGET_TABLE_NAME')
            if not table_name:
                continue
                
            total_list.append(table_name)
            
            # Verificar si la tabla necesita ser añadida al crawler
            if 'CRAWLER' not in item or not item['CRAWLER']:
                empty_tables.append(table_name)
                tables_to_update.append(table_name)
            
            # Preparar delta target para esta tabla
            if 'STAGE_TABLE_NAME' in item:
                data_source_target = {
                    'DeltaTables': [f"{s3_target}{team}/{data_source}/{item['ENDPOINT']}/{item['STAGE_TABLE_NAME']}/"],
                    'ConnectionName': '',
                    'CreateNativeDeltaTable': True
                }
                delta_targets.append(data_source_target)
        
        # Actualizar todas las tablas vacías en una sola operación por lotes (si es posible)
        if tables_to_update:
            # Aquí podríamos implementar una operación por lotes si usamos batch_write_item
            for table in tables_to_update:
                glue_crawler_manager.update_dynamodb_attribute(
                    config_table, 'TARGET_TABLE_NAME', table, 'CRAWLER', True
                )
                logger.info(f"Tabla {table} añadida al crawler")
                
    except Exception as e:
        logger.error(f"Error al procesar tablas para el endpoint {endpoint_name}: {str(e)}")
    
    return total_list, empty_tables, delta_targets

def main():
    """Función principal"""
    try:        
        # Inicializar el gestor de Glue
        glue_crawler_manager = GlueCrawlerManager(logger)
        
        # Variables principales
        dynamo_config_table = args['DYNAMO_CONFIG_TABLE']
        dynamo_endpoint_table = args['DYNAMO_ENDPOINT_TABLE']
        s3_target = args['S3_STAGE_PREFIX']
        arn_role_crawler = args['ARN_ROLE_CRAWLER']
        job_name = args['JOB_NAME']
        endpoint_name = args['ENDPOINT']
        team = args['TEAM']
        data_source = args['DATA_SOURCE']
        process_id = args['PROCESS_ID']
        
        # Obtener datos del endpoint (una sola llamada a DynamoDB)
        endpoint_data = glue_crawler_manager.get_dynamodb_item(dynamo_endpoint_table, 'ENDPOINT_NAME', endpoint_name)

        if not endpoint_data:
            logger.error(f"No se encontraron datos para el endpoint {endpoint_name}")
            return
        
        # Definir nombres de recursos
        data_catalog_database_name = f"{team}_{data_source}_{endpoint_name}_stage".lower()
        data_catalog_crawler_name = data_catalog_database_name + "_crawler"
        
        logger.info(f"Procesando endpoint: {endpoint_name}")
        logger.info(f"Base de datos: {data_catalog_database_name}")
        logger.info(f"Crawler: {data_catalog_crawler_name}")
        
        # Obtener tablas y preparar targets en una sola operación (optimizado)
        total_list, empty_tables, delta_targets = get_tables_and_prepare_delta_targets(
            glue_crawler_manager, dynamo_config_table, endpoint_name, 
            s3_target, team, data_source
        )
        
        if not total_list:
            logger.warning(f"No se encontraron tablas para el endpoint {endpoint_name}")
            return
            
        logger.info(f"Total de tablas: {len(total_list)}, Tablas sin crawler: {len(empty_tables)}")
        
        if not delta_targets:
            logger.error("No se pudieron preparar objetivos Delta para el crawler")
            return
        
        # Comprobar si existe el crawler
        if glue_crawler_manager.crawler_exists(data_catalog_crawler_name):
            if empty_table:
                logger.info(f"Actualizando crawler con {len(empty_table)} nuevas tablas")
                glue_crawler_manager.update_delta_crawler(
                    data_catalog_crawler_name, arn_role_crawler, 
                    data_catalog_database_name, delta_targets
                )
            
            logger.info(f"Iniciando crawler existente {data_catalog_crawler_name}")
            glue_crawler_manager.start_crawler(data_catalog_crawler_name)
        else:
            logger.info("Verificando existencia de la base de datos en el catálogo")
            
            if glue_crawler_manager.database_exists(data_catalog_database_name):
                logger.info(f"Creando nuevo crawler para base de datos existente {data_catalog_database_name}")
                glue_crawler_manager.create_delta_crawler(
                    data_catalog_crawler_name, arn_role_crawler, 
                    data_catalog_database_name, delta_targets
                )
                glue_crawler_manager.start_crawler(data_catalog_crawler_name)
            else:
                logger.info("Obteniendo el ARN del rol del trabajo")
                logger.info(f"Glue Job: {job_name}")
                job_role_arn = glue_crawler_manager.get_job_role_arn(job_name)
                
                if not job_role_arn:
                    logger.error(f"No se pudo obtener el ARN del rol para el trabajo {job_name}")
                    return
                
                logger.info("Asignando permisos de etiquetas LF")
                glue_crawler_manager.grant_lf_tag_permissions(job_role_arn)
                
                logger.info(f"Creando base de datos {data_catalog_database_name}")
                if not glue_crawler_manager.create_database(data_catalog_database_name):
                    logger.error("No se pudo crear la base de datos. Abortando.")
                    return
                
                logger.info("Añadiendo etiquetas LF a la base de datos")
                glue_crawler_manager.add_lf_tags_to_database(data_catalog_database_name)
                
                logger.info("Asignando permisos a la base de datos")
                glue_crawler_manager.grant_database_permissions(job_role_arn, data_catalog_database_name)
                
                logger.info("Creando nuevo crawler")
                if glue_crawler_manager.create_delta_crawler(
                    data_catalog_crawler_name, arn_role_crawler, 
                    data_catalog_database_name, delta_targets
                ):
                    logger.info("Iniciando nuevo crawler")
                    glue_crawler_manager.start_crawler(data_catalog_crawler_name)
                else:
                    logger.error("No se pudo crear el crawler. Abortando.")
                    
        logger.info("Proceso completado exitosamente")
    except Exception as e:
        logger.error(f"Error en la función principal: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    main()

2025-05-18 15:52:41,066 botocore.credentials INFO Found credentials in shared credentials file: ~/.aws/credentials
2025-05-18 15:52:42,152 root INFO Procesando endpoint: PEBDDATA
2025-05-18 15:52:42,153 root INFO Base de datos: sofia_bigmagic_pebddata_stage
2025-05-18 15:52:42,154 root INFO Crawler: sofia_bigmagic_pebddata_stage_crawler
2025-05-18 15:52:43,035 root INFO Total de tablas: 106, Tablas sin crawler: 0
2025-05-18 15:52:44,281 root INFO Crawler sofia_bigmagic_pebddata_stage_crawler no encontrado
2025-05-18 15:52:44,282 root INFO Verificando existencia de la base de datos en el catálogo
2025-05-18 15:52:44,516 root INFO Base de datos sofia_bigmagic_pebddata_stage encontrada
2025-05-18 15:52:44,518 root INFO Creando nuevo crawler para base de datos existente sofia_bigmagic_pebddata_stage
2025-05-18 15:52:44,998 root INFO Crawler sofia_bigmagic_pebddata_stage_crawler creado exitosamente
2025-05-18 15:52:56,842 root INFO Crawler sofia_bigmagic_pebddata_stage_crawler iniciado exit